# Trabajo matematicas Aplicadas a Sistemas de Información.
## Problemas de Asignación; Método Húngaro
### Autores: Guillermo Ramón Soria y David Ruiz Jurado

 

### La función se puede entender mejor separandola en dos pasos:
<ul>
<li> La función se utiliza para encontrar la fila que contiene el menor número de 0.</li> 
<li> Selecciona el número cero en la fila, y luego marca el elemento correspondiente a la fila y columna como Falso</li> 
<ul>



In [3]:
import numpy as np

In [4]:
def fila_de_minimos_ceros(matriz_ceros, ceros_marcados):
    
	#Encontramos la fila
	fila_minima = [99999, -1]
	for fila in range(matriz_ceros.shape[0]): 
		if np.sum(matriz_ceros[fila] == True) > 0 and fila_minima[0] > np.sum(matriz_ceros[fila] == True):
			fila_minima = [np.sum(matriz_ceros[fila] == True), fila]

	# Marcamos la posicion de los ceros como falsa
	zero_index = np.where(matriz_ceros[fila_minima[1]] == True)[0][0]
	ceros_marcados.append((fila_minima[1], zero_index))
	matriz_ceros[fila_minima[1], :] = False
	matriz_ceros[:, zero_index] = False


Encontrar las posibles soluciones para el problema de asignación lineal

In [24]:
def matriz_marca(matriz):

    #Se transforma la matriz a booleana donde (0 = True, others = False)
    matriz_actual = matriz
    matriz_ceros_booleanos = (matriz_actual == 0)
    matriz_ceros_booleanos_copy = matriz_ceros_booleanos.copy()
		
    #Se guardan posibles soluciones en funcion de la variable ceros_marcados
    ceros_marcados = []
    while (True in matriz_ceros_booleanos_copy):
        fila_de_minimos_ceros(matriz_ceros_booleanos_copy, ceros_marcados)

    #Se guardan filas y columnas de manera independiente
    ceros_marcados_fila = []
    ceros_marcados_columna = []
    for i in range(len(ceros_marcados)):
        ceros_marcados_fila.append(ceros_marcados[i][0])
        ceros_marcados_columna.append(ceros_marcados[i][1])
    #paso 2-2-1
    filas_no_marcadas = list(set(range(matriz_actual.shape[0])) - set(ceros_marcados_fila))
    
    columnas_marcadas = []
    comprobante = True
    while comprobante:
        comprobante = False
        for i in range(len(filas_no_marcadas)):
            matriz_fila = matriz_ceros_booleanos[filas_no_marcadas[i], :]
            for j in range(matriz_fila.shape[0]):
                #paso 2-2-2
                if matriz_fila[j] == True and j not in columnas_marcadas:
                    #paso 2-2-3
                    columnas_marcadas.append(j)
                    comprobante = True

        for fila, columna in ceros_marcados:
            #paso 2-2-4
            if fila not in filas_no_marcadas and columna in columnas_marcadas:
                #paso 2-2-5
                filas_no_marcadas.append(fila)
                comprobante = True
    #paso 2-2-6
    filas_marcadas = list(set(range(matriz.shape[0])) - set(filas_no_marcadas))
    
    return(ceros_marcados, filas_marcadas, columnas_marcadas)

Ajuste de matriz

In [25]:
def matriz_de_ajuste(mat, recubrimiento_filas, recubrimiento_columnas):
	matriz_actual = mat
	valores_no_cero = []

	#paso 4-1
	for fila in range(len(matriz_actual)):
		if fila not in recubrimiento_filas:
			for i in range(len(matriz_actual[fila])):
				if i not in recubrimiento_columnas:
					valores_no_cero.append(matriz_actual[fila][i])
	mininimo = min(valores_no_cero)

	#paso 4-2
	for fila in range(len(matriz_actual)):
		if fila not in recubrimiento_filas:
			for i in range(len(matriz_actual[fila])):
				if i not in recubrimiento_columnas:
					matriz_actual[fila, i] = matriz_actual[fila, i] - mininimo
	#paso 4-3
	for fila in range(len(recubrimiento_filas)):  
		for columna in range(len(recubrimiento_columnas)):
			matriz_actual[recubrimiento_filas[fila], recubrimiento_columnas[columna]] = matriz_actual[recubrimiento_filas[fila], recubrimiento_columnas[columna]] + mininimo
	return matriz_actual

In [26]:

def algoritmo_hungaro(mat): 
	dim = mat.shape[0]
	matriz_actual = mat

	for fila in range(mat.shape[0]): 
		matriz_actual[fila] = matriz_actual[fila] - np.min(matriz_actual[fila])
	
	for columna in range(mat.shape[1]): 
		matriz_actual[:,columna] = matriz_actual[:,columna] - np.min(matriz_actual[:,columna])
	cuenta = 0
	while cuenta < dim:
		#paso 2 y 3
		posiciones_respuesta, filas_marcadas, columnas_marcadas = matriz_marca(matriz_actual)
		cuenta = len(filas_marcadas) + len(columnas_marcadas)

		if cuenta < dim:
			matriz_actual = matriz_de_ajuste(matriz_actual, filas_marcadas, columnas_marcadas)

	return posiciones_respuesta


In [15]:
def calculo_solucion(mat, pos):
	total = 0
	matriz_solucion = np.zeros((mat.shape[0], mat.shape[1]))
	for i in range(len(pos)):
		total += mat[pos[i][0], pos[i][1]]
		matriz_solucion[pos[i][0], pos[i][1]] = mat[pos[i][0], pos[i][1]]
	return total, matriz_solucion

# Algoritmo Húngaro: 
Finding the minimum value in linear assignment problem.
Therefore, we can find the minimum value set in net matrix 
by using Hungarian Algorithm. In other words, the maximum value
and elements set in cost matrix are available.

In [27]:
import numpy as np
# La matriz que quieres encontrar la suma minima:
cost_matrix = np.array([
    [4,2,5,7],
    [8,3,10,8],
    [12,5,4,5],
    [6,3,7,14]
])
posicion_respuesta = algoritmo_hungaro(cost_matrix.copy())
respuesta, matriz_respuesta = calculo_solucion(cost_matrix, posicion_respuesta)

print(f"Resultado del problemas de asignacion: {respuesta:.0f}\n\nMatriz:\n{matriz_respuesta}")


Resultado del problemas de asignacion: 19

Matriz:
[[4. 0. 0. 0.]
 [0. 3. 0. 0.]
 [0. 0. 0. 5.]
 [0. 0. 7. 0.]]
